In [8]:
import matplotlib.pyplot as plt
import numpy as np


def plot_return_condensed(CM, x_train, y_train, verbose=1):

    plt.figure()

    x_train_condensed, y_train_condensed = CM._fit_transform(x_train, y_train)
    if verbose >= 1:
        print(
            f"Selected indices ({len(x_train_condensed)}): {CM._get_selected_indices()}"
        )
        print(y_train_condensed)
        print(f"Labels: {len(np.unique(y_train_condensed))/len(np.unique(y_train))}")

    _, _, len_ts = x_train_condensed.shape

    color = {0: "black", 1: "blue", 2: "red", 3: "green", 4: "pink", 5: "orange"}

    x = range(0, len_ts)

    for idx, (series, label) in enumerate(zip(x_train, y_train)):
        alpha = 0.2
        if idx in CM._get_selected_indices():
            alpha = 1

        plt.plot(x, series[0, :], color=color[int(label)], alpha=alpha)

    return x_train_condensed, y_train_condensed

In [2]:
from aeon.datasets._single_problem_loaders import (
    load_arrow_head,
    load_gunpoint,
    load_unit_test,
)
from sklearn.metrics import accuracy_score

from tsml_eval._wip.condensing.drop1 import Drop1
from tsml_eval._wip.condensing.simple_rank import SimpleRank
from tsml_eval._wip.condensing.wrapper import WrapperBA
from tsml_eval.estimators.classification.distance_based import (
    KNeighborsTimeSeriesClassifier,
)

for dataset in [load_unit_test, load_arrow_head, load_gunpoint]:

    print(dataset)

    x_train, y_train = dataset("TRAIN")

    CM = SimpleRank(distance="msm", n_neighbors=2)  # SR
    CM = Drop1(distance="msm", n_neighbors=2)  # Drop1
    CM = WrapperBA(distance="msm")  # MBA/DBA

    # x_train_condensed, y_train_condensed = plot_return_condensed(
    #   CM, x_train, y_train, 0
    # )
    x_train_condensed, y_train_condensed = CM._fit_transform(x_train, y_train)

    x_test, y_test = dataset("TEST")

    knn = KNeighborsTimeSeriesClassifier(distance="msm", n_neighbors=1)
    knn.fit(x_train, y_train)
    y_pred_full = knn.predict(x_test)

    knn.fit(x_train_condensed, y_train_condensed)
    y_pred_cond = knn.predict(x_test)

    print(f"\tFull: {accuracy_score(y_test, y_pred_full)}")
    print(f"\tCondensed: {accuracy_score(y_test, y_pred_cond)}")

<function load_unit_test at 0x7fa5641bd000>
	Full: 0.9090909090909091
	Condensed: 0.9090909090909091
<function load_arrow_head at 0x7fa5641bd090>
	Full: 0.7942857142857143
	Condensed: 0.5371428571428571
<function load_gunpoint at 0x7fa5641bcdc0>
	Full: 0.9666666666666667
	Condensed: 0.7066666666666667
